In [1]:
import pandas as pd
import geopandas as gpd
import mio
import numpy as np
from matplotlib import pyplot as plt
import pathlib
import time

In [2]:
def tri_array(p0, p1, p2, area, px, py):
    
    r_shape = tri_shape(p0, p1, p2, area, px, py)
    r_heights = tri_heights(p0, p1, p2, area, px, py)
    
    return r_shape * r_heights

In [3]:
def tri_heights(p0, p1, p2, area, px, py):
    # calculate the triangle plane
    Ax, Ay, Az = p0
    Bx, By, Bz = p1
    Cx, Cy, Cz = p2
    
    AB = (Bx - Ax, By - Ay, Bz - Az)
    AC = (Cx - Ax, Cy - Ay, Cz - Az)
    
    # AB x AC = (a,b,c)
    # Plane equasion: ax + by + cz + d = 0
    a, b, c = np.cross(AB, AC)
    d = -(a * Ax + b * Ay + c * Az)
 
    pz  = (-a * px - b * py - d) / c

    return pz

In [4]:
def tri_shape(p0, p1, p2, area, px, py):
    # calc the outline of the triangles
    p0x, p0y, _ = p0
    p1x, p1y, _ = p1
    p2x, p2y, _ = p2
    
    # do the calc, using stolen formula
    s = 1/(2*area)*(p0y*p2x - p0x*p2y + (p2y - p0y)*px + (p0x - p2x)*py)
    t = 1/(2*area)*(p0x*p1y - p0y*p1x + (p0y - p1y)*px + (p1x - p0x)*py)
    u = 1 - s -t
    r = (s>0) & (t>0) & (u>0)
    
    return r.astype(int)

In [5]:
def signed_area(p0, p1, p2):
        # calc signed area of triangle
        p0x, p0y, _ = p0
        p1x, p1y, _ = p1
        p2x, p2y, _ = p2
        
        a = 0.5 *(-p1y*p2x + p0y*(-p1x + p2x) + p0x*(p1y - p2y) + p1x*p2y)

        return a

In [6]:
def create_meshgrid(p0, p1, p2):
    xmin = int(min(p0[0], p1[0], p2[0])) -1
    xmax = int(max(p0[0], p1[0], p2[0])) +1
    ymin = int(min(p0[1], p1[1], p2[1])) -1
    ymax = int(max(p0[1], p1[1], p2[1])) +1
    df = pd.DataFrame(
        columns = range(xmin, xmax, 1),
        index = range(ymax, ymin, -1),
        data = 0
    )
    xa = np.arange(xmin, xmax,  1)
    ya = np.arange(ymax, ymin, -1)
    px, py = np.meshgrid(xa, ya)
    return xa, ya, px, py

In [7]:
def get_df_list(tab):
    print(f'reading {tab}', end=' ')
    vec = gpd.read_file(tab)
    ldf = []
    count_ok = 0
    count_er = 0
    
    i = 0
    print(len(vec) / 1e3, 'x 1000 triangles')
    print('create mini rasters')
    for ind, row in vec.iterrows():
        i += 1
        mio.show_perc(i, len(vec), 10000)
        p0 = (row.x0, row.y0, row.z0)
        p1 = (row.x1, row.y1, row.z1)
        p2 = (row.x2, row.y2, row.z2)

        xa, ya, px, py = create_meshgrid(p0, p1, p2)

        area = signed_area(p0, p1, p2)

        # get rid of mini areas
        if area >= 0.5:
            count_ok += 1
            ar = tri_array(p0, p1, p2, area, px, py)
            df = pd.DataFrame(ar, columns=xa, index=ya)
            if len(df) > 0:
                ldf.append(df)
        else:
            # propably vertical
            count_er += 1
    return ldf

In [8]:
def get_big_empty(ldf):
    lx, ly = [], []
    for df in ldf:
        lx.append(df.columns.min())
        lx.append(df.columns.max())
        ly.append(df.index.min())
        ly.append(df.index.max())
    xmin, xmax = min(lx)-5, max(lx)+5
    ymin, ymax = min(ly)-5, max(ly)+5
    
    empty = pd.DataFrame(
        columns = range(xmin, xmax, 1),
        index = range(ymax, ymin, -1),
        data = 0
    )
    return empty

In [11]:
def get_tag_todo():
    source_dir = pathlib.Path('tab')
    tags_all = set([d.stem for d in source_dir.glob('*.tab')])

    dest_dir = pathlib.Path('tif')
    tags_done =  set([t.stem[0:7] for t in dest_dir.glob('*.tif')])
    #print(tags_done)

    tags_todo = tags_all.difference(tags_done)
    #print(tags_todo)
    l = list(tags_todo)
    l.sort()
    if len(l) >0:
        print(f"{len(l)} files to do")
        return l[0]
    else:
        return

In [12]:
get_tag_todo()

1 files to do


'1091-41'

In [ ]:
# MAIN
while True:
    tag = get_tag_todo()
    if tag is not None:
        # Create list of mini dataframes
        ldf = get_df_list(f'tab/{tag}.tab')

        # Create empty raster
        big = get_big_empty(ldf)

        print('\ncombining all the mini rasters')
        for i, small in enumerate(ldf):
            mio.show_perc(i, len(ldf), 5_000)
            xmin, xmax = small.columns.min(), small.columns.max()
            ymin, ymax = small.index.min(), small.index.max()
            big.loc[ymax:ymin, xmin:xmax] = np.maximum(big.loc[ymax:ymin, xmin:xmax], small)

        # Save raster file
        print(f'writing {tag}_raster.tif')
        mio.write_raster(big.fillna(0).astype('int16'), f'tif/{tag}_raster.tif')
        time.sleep(10)

1 files to do
reading tab/1091-41.tab 2252.844 x 1000 triangles
create mini rasters
0.44% 0.89% 1.33% 1.78% 2.22% 2.66% 3.11% 3.55% 3.99% 4.44% 4.88% 5.33% 5.77% 6.21% 6.66% 7.1% 7.55% 7.99% 8.43% 8.88% 9.32% 9.77% 10.21% 10.65% 11.1% 11.54% 11.98% 12.43% 12.87% 13.32% 13.76% 14.2% 14.65% 15.09% 15.54% 15.98% 16.42% 16.87% 17.31% 17.76% 18.2% 18.64% 19.09% 19.53% 19.97% 20.42% 20.86% 21.31% 21.75% 22.19% 22.64% 23.08% 23.53% 23.97% 24.41% 24.86% 25.3% 25.75% 26.19% 26.63% 27.08% 27.52% 27.96% 28.41% 28.85% 29.3% 29.74% 30.18% 30.63% 31.07% 31.52% 31.96% 32.4% 32.85% 33.29% 33.74% 34.18% 34.62% 35.07% 35.51% 35.95% 36.4% 36.84% 37.29% 37.73% 38.17% 38.62% 39.06% 39.51% 39.95% 40.39% 40.84% 41.28% 41.73% 42.17% 42.61% 43.06% 43.5% 43.94% 44.39% 44.83% 45.28% 45.72% 46.16% 46.61% 47.05% 47.5% 47.94% 48.38% 48.83% 49.27% 49.71% 50.16% 50.6% 51.05% 51.49% 51.93% 52.38% 52.82% 53.27% 53.71% 54.15% 54.6% 55.04% 55.49% 55.93% 56.37% 56.82% 57.26% 57.7% 58.15% 58.59% 59.04% 59.48% 59.92% 60.37%